In [1]:
import requests as r
from bs4 import BeautifulSoup
import re

In [2]:
url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-11-08&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"
res = r.get(url)
html = res.text
bf = BeautifulSoup(html)

In [10]:
searchpart = bf.find('ol',{'data-testid':re.compile("search-result")})
items = searchpart.find_all('li',{'data-testid':re.compile("search-bodega-result")})

In [12]:
print(items[0])

<li class="css-1l4w6pd" data-testid="search-bodega-result"><div class="css-1bdu3ax"><span class="css-17ubb9w"> </span><div class="css-1i8vfl5"><div class="css-e1lvw9"><p class="css-myxawk">Politics</p><a href="/2024/11/08/us/politics/harris-trump-democrats-election.html?searchResultPosition=1"><h4 class="css-nsjm9t">Democrats Consider an Autopsy, but Who’s the Coroner?</h4><p class="css-16nhkrn">The Democratic Party agrees it needs to figure out what went wrong. The question is how.</p><p class="css-1engk30">By Jess Bidgood and Adam Nagourney</p></a></div><figure aria-label="media" class="css-tap2ym" role="group"><div data-testid="imageContainer-children-Image"><img alt="A campaign sign in Bozeman, Mont., last month." class="css-rq4mmj" decoding="async" height="1" sizes="(max-width: 600px) 120px, (max-width: 1024px) 165px, 205px" src="https://static01.nyt.com/images/2024/11/08/multimedia/08-pol-on-politics-newsletter-women-topitem-khpq/08-pol-on-politics-newsletter-women-topitem-khpq-t

In [106]:
def getDateFromHref(href):
    tmp = href.split('/')
    if len(tmp)<4:
        return 'UnKnown'
    year = tmp[1]
    month = tmp[2]
    day = tmp[3]
    if(tmp[1]=="live"):
        year = tmp[2]
        month = tmp[3]
        day = tmp[4]
    
    return year+'-'+month+'-'+day

In [42]:
def extractItem(item):
    # time = item.find('span',{'data-testid':re.compile("todays-date")})
    content = item.find('a',href=True)
    href = content['href']
    date = getDateFromHref(href)
    link = 'https://www.nytimes.com'+href
    title = content.find('h4').text
    summary = content.find('p').text
    obj = {
        'Title': title,
        'Summary': summary,
        'Date': date,
        'Url': link
    }
    return obj

In [43]:
obj = extractItem(items[2])

In [44]:
print(obj)

{'Title': 'San Francisco Mayor-Elect Plans to Declare Fentanyl Emergency on Day 1', 'Summary': 'Daniel Lurie held his first news conference since Mayor London Breed conceded the mayoral race this week.', 'Date': '2024-11-08', 'Url': 'https://www.nytimes.com/2024/11/08/us/elections/san-francisco-daniel-lurie-fentanyl.html?searchResultPosition=3'}


In [57]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait

In [71]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC  

In [58]:
driver = webdriver.Firefox()

In [114]:
# url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-11-08&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"
# url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-09-30&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"
# url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-08-08&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"
# url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-06-26&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"
# url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-04-10&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"
url = r"https://www.nytimes.com/search?dropmab=false&endDate=2024-02-01&lang=en&query=2024%20US%20Elections&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=newest&startDate=2024-01-01&types=article"

page = driver.get(url)
wait = WebDriverWait(driver, 10)

In [115]:
bf = BeautifulSoup(driver.page_source)

In [116]:
searchpart = bf.find('ol',{'data-testid':re.compile("search-result")})
items = searchpart.find_all('li',{'data-testid':re.compile("search-bodega-result")})

In [117]:
obj_list = []
cnt = 0
for item in items:
    obj = extractItem(item)
    obj_list.append(obj)
    cnt = cnt + 1

In [119]:
for i in range(100):
    button = driver.find_element(By.XPATH, "//button[@data-testid='search-show-more-button']")
    button.click()

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//li[@data-testid='search-bodega-result']")))
    new_bf = BeautifulSoup(driver.page_source)
    new_searchpart = new_bf.find('ol',{'data-testid':re.compile("search-result")})
    all_items = new_searchpart.find_all('li',{'data-testid':re.compile("search-bodega-result")})
    new_items = all_items[cnt:]
    for item in new_items:
        obj = extractItem(item)
        obj_list.append(obj)
        cnt = cnt + 1
    print(len(obj_list))

NoSuchElementException: Message: Unable to locate element: //button[@data-testid='search-show-more-button']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [120]:
import json
# with open("2024+US+Elections_10-11_NYT.json", "w", encoding='utf-8') as file:
# with open("2024+US+Elections_8-9_NYT.json", "w", encoding='utf-8') as file:
# with open("2024+US+Elections_7_NYT.json", "w", encoding='utf-8') as file:
# with open("2024+US+Elections_4-6_NYT.json", "w", encoding='utf-8') as file:
with open("2024+US+Elections_1_NYT.json", "w", encoding='utf-8') as file:
    json.dump(obj_list, file, indent=4, ensure_ascii=False)

In [121]:
all_obj = []

In [132]:
def checkDate(date,url):
    dates = date.split('-')
    rs = date
    if(dates[0]=="live"):
        urls = url.split('/')
        year = urls[4]
        month = urls[5]
        day = urls[6]
        rs = year+"-"+month+"-"+day
    return rs

In [136]:
with open("2024+US+Elections_10-11_NYT.json", "r", encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        title = item['Title']
        summary = item['Summary']
        date = item['Date']
        url = item['Url']
        date = checkDate(date,url)
        obj = {
            'Title': title,
            'Summary': summary,
            'Date': date,
            'Url': url
        }
        all_obj.append(obj)

In [137]:
with open("2024+US+Elections_8-9_NYT.json", "r", encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        title = item['Title']
        summary = item['Summary']
        date = item['Date']
        url = item['Url']
        date = checkDate(date,url)
        obj = {
            'Title': title,
            'Summary': summary,
            'Date': date,
            'Url': url
        }
        all_obj.append(obj)

In [138]:
with open("2024+US+Elections_7_NYT.json", "r", encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        title = item['Title']
        summary = item['Summary']
        date = item['Date']
        url = item['Url']
        date = checkDate(date,url)
        obj = {
            'Title': title,
            'Summary': summary,
            'Date': date,
            'Url': url
        }
        all_obj.append(obj)

In [139]:
with open("2024+US+Elections_4-6_NYT.json", "r", encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        title = item['Title']
        summary = item['Summary']
        date = item['Date']
        url = item['Url']
        date = checkDate(date,url)
        obj = {
            'Title': title,
            'Summary': summary,
            'Date': date,
            'Url': url
        }
        all_obj.append(obj)

In [140]:
with open("2024+US+Elections_2-3_NYT.json", "r", encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        title = item['Title']
        summary = item['Summary']
        date = item['Date']
        url = item['Url']
        date = checkDate(date,url)
        obj = {
            'Title': title,
            'Summary': summary,
            'Date': date,
            'Url': url
        }
        all_obj.append(obj)

In [141]:
with open("2024+US+Elections_1_NYT.json", "r", encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        title = item['Title']
        summary = item['Summary']
        date = item['Date']
        url = item['Url']
        date = checkDate(date,url)
        obj = {
            'Title': title,
            'Summary': summary,
            'Date': date,
            'Url': url
        }
        all_obj.append(obj)

In [142]:
with open("2024+US+Elections_NYT.json", "w", encoding='utf-8') as file:
    json.dump(all_obj, file, indent=4, ensure_ascii=False)